In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [1]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.linear_model import LogisticRegression
import pandas as pd 
from sklearn.model_selection import train_test_split

with mlflow.start_run():
    # load dataset and train model
    df = pd.read_csv('Admission_Predict.csv')
    x = df[['gre', 'sop', 'cgpa']]
    y = df['admitted']

    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    C = 1
    intercept_scaling = 1

    def eval_metrics(actual, pred):
        acc_score = accuracy_score(actual, pred)
        pre_score = precision_score(actual, pred)
        f_score = f1_score(actual, pred)

        return acc_score, pre_score, f_score

    lr = LogisticRegression(C=C, intercept_scaling=intercept_scaling)
    prediction = lr.fit(x_train, y_train)

    #log metrics
    (acc_score, pre_score, f_score) = eval_metrics(y_test, prediction)
    print('Logistic Regression (C={:f}, intercept_scaling{:f}:'.format(C, intercept_scaling))
    print("accuracy score: %s" % acc_score)
    print("precision score score: %s" % pre_score)
    print("f1 score: %s" % f_score)

    mlflow.log_metric('accuracy score', acc_score)
    mlflow.log_metric('precision score', pre_score)
    mlflow.log_metric('f1 score', f_score)

    # log model params
    mlflow.log_param("C", lr.C)
    mlflow.log_param("Intercept Scaling", lr.intercept_scaling)
    signature = infer_signature(x_train, lr.predict(x_train))

    # log model
    mlflow.sklearn.log_model(lr, "lr_models", signature=signature)

c:\Users\TeeFaith\anaconda3\envs\kenny\Lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
